# Webmining - Assignment 2

This **Home Assignment** is to be submitted and you will be given points for each of the tasks. It familiarizes you with basics of Markov Chain and link prediction on graphs.

## Formalities
**Submit in a group of 2-3 people until 16.06.2020 23:59CET. The deadline is strict!**

## Evaluation and Grading
General advice for programming excercises at *CSSH*:
Evaluation of your submission is done semi automatically. Think of it as this notebook being 
executed once. Afterwards, some test functions are appended to this file and executed respectively.

Therefore:
* Submit valid _Python3_ code only!
* Use external libraries only when specified by task.
* Ensure your definitions (functions, classes, methods, variables) follow the specification if
  given. The concrete signature of e.g. a function usually can be inferred from task description, 
  code skeletons and test cases.
* Ensure the notebook does not rely on current notebook or system state!
  * Use `Kernel --> Restart & Run All` to see if you are using any definitions, variables etc. that 
    are not in scope anymore.
  * Double check if your code relies on presence of files or directories other than those mentioned
    in given tasks. Tests run under Linux, hence don't use Windows style paths 
    (`some\path`, `C:\another\path`). Also, use paths only that are relative to and within your
    working directory (OK: `some/path`, `./some/path`; NOT OK: `/home/alice/python`, 
    `../../python`).
* Keep your code idempotent! Running it or parts of it multiple times must not yield different
  results. Minimize usage of global variables.
* Ensure your code / notebook terminates in reasonable time.

**There's a story behind each of these points! Don't expect us to fix your stuff!**

Regarding the scores, you will get no points for a task if:
- your function throws an unexpected error (e.g. takes the wrong number of arguments)
- gets stuck in an infinite loop
- takes much much longer than expected (e.g. >1s to compute the mean of two numbers)
- does not produce the desired output (e.g. returns an descendingly sorted list even though we asked for ascending, returns the mean and the std even though we asked for only the mean, prints an output instead of returning it, ...)

In [1]:
# credentials of all team members (you may add or remove items from the dictionary)
team_members = [
    {
        'first_name': 'Pavel',
        'last_name': 'Raschetnov',
        'student_id': 404839
    },
    {
        'first_name': 'Anya',
        'last_name': 'Poudyal',
        'student_id': 391805
    },
    {
        'first_name': 'Philipp',
        'last_name': 'Stein',
        'student_id': 397615
    }
]

# Task 1: Markov chains (6 points total)

For this task you are only allowed to use numpy and pandas. For this task always include the RESET states.

### Information: data structures for this assignment:
A *state* or *item* is always given by a string.
A uni-variate sequence is given (or should be implemented as) a list of states (strings).

Example:

```uni_variate_sequence = ["a", "c", "b", "a"]```

```list_of_sequences =  [["a"], ["a", "b", "c"], ["a", "c"], ["c"]]```

We usually refer to a list of sequences as just `sequences`

## 1a) Transitions generator (1)
Write a function `generate_transitions(sequences, order, include_RESET=True)` that loops over each of the sequences in the provided list of sequences and `yield`s one transition after another. A transition is a tuple where the first element is a tuple containing the relevant history (i.e. one element for order 1, two elements for order two ...). The second element is the next state. Whether reset states should be included can be specified through `include_RESET`. Use `"RESET"` as the indicating string of the reset state (you can assume this string does not occur in the data otherwise). If a sequence is shorter than order + 1 and include_RESET = False, omit that sequence.

Example:
```python
test_sequences = [["a", "b"]]
list(generate_transitions(test_sequences, order))
# order=1 [(('RESET',), 'a'), (('a',), 'b'), (('b',), 'RESET')]
# order=2 [(('RESET', 'RESET'), 'a'), (('RESET', 'a'), 'b'), (('a', 'b'), 'RESET')]
test_sequences = [["a"], ["b"]]
list(generate_transitions(test_sequences, 1))
# order=1 [(('RESET',), 'a'), (('a',), 'RESET'), (('RESET',), 'b'), (('b',), 'RESET')]
```


## 1b) Fitting the model (1)
Implement the function `fit_mc(sequences, order)` that fits a Markov chain of a specific `order` to the given input of a list of sequences.
The function should return t_m, row_names, col_names with

* `t_m`: transition_probability matrix (2-d numpy array of type float)
* `row_names`: List of source state tuples in the ordering used in the array. 
* `col_names`: List of the target states in the ordering used in the array

Do not include empty rows i.e. rows that have not been observed.


Example:
```python
t_m = np.array([[0.0, 0.5, 0.5],
                [0.2, 0.8, 0.0],
                [0.3, 0.4, 0.4]])
row_names = ["RESET", "state_a", "state_b"]
col_names = [("RESET",), ("state_a",), ("state_b",)]
```
means that the transition probability to go from "state_a" to "RESET" is 0.2

In [2]:
import numpy as np
import pandas as pd
from collections import defaultdict

In [3]:
DEBUG=True

In [4]:
def generate_transitions(sequences, order, include_RESET=True):
    for s in sequences:
        if include_RESET:
            s = ['RESET'] * order + s + ['RESET']
        for i in range(len(s) - order):
            yield (tuple(s[i:i+order]), s[i+order])

In [5]:
def fit_mc(sequences, order):
    counts = defaultdict(int)
    row_names = {}
    col_names = {}

    for source,target in generate_transitions(sequences, order):
        i,j = len(row_names),len(col_names)
        if source not in row_names:
            row_names[source] = i
        if target not in col_names:
            col_names[target] = j
            
        counts[(row_names[source],col_names[target])] += 1
    
#     print(row_names)
#     print(col_names)
#     print(counts)
    
    t_m = np.zeros(shape=(len(row_names),len(col_names)), dtype=np.float)
    for i,j in counts.keys():
        t_m[i,j] = counts[(i,j)]
    t_m /= t_m.sum(axis=1)[:,np.newaxis]
    
    return t_m, list(row_names.keys()), list(col_names.keys())

## 1c) Comparing Markov Models (1.5)
Implement three functions
* `log_likelihood(sequences, t_m, row_names, col_names)`
* `AIC(sequences, t_m, row_names, col_names)`
* `BIC(sequences, t_m, row_names, col_names)`

That compute the log likelihood, AIC and BIC scores for the provided sequences given the markov model specified through `t_m`, `row_names`, `col_names`. Use the natural logarithm.
When calculating the log_likelihood skip transitions that have not been observed.

In [6]:
def get_k(n_states, order):
    k = (n_states ** order - n_states ** (order - 1) + 1) * (n_states - 1)
    k -= 1
    return k

def log_likelihood(sequences, t_m, row_names, col_names):
    order = len(next(iter(row_names)))
    transitions = generate_transitions(sequences, order)
    
    ll = 0
    for source,target in transitions:
#         print(source,target)
#         print(row_names)
        i,j = row_names.index(source),col_names.index(target)
        ll += np.log(t_m[i,j])
    return ll
    
    
def AIC(sequences, t_m, row_names, col_names):
    order = len(next(iter(row_names)))
    
    n_rows, n_cols = len(row_names), len(col_names)
    k = get_k(len(col_names), order)
    
    return 2 * k - 2 * log_likelihood(sequences, t_m, row_names, col_names)
    
    
    
def BIC(sequences, t_m, row_names, col_names):
    order = len(next(iter(row_names)))
    transitions = generate_transitions(sequences, order)
    n = len(list(transitions))
    
    k = get_k(len(col_names), order)
    
    return np.log(n) * k - 2 * log_likelihood(sequences, t_m, row_names, col_names)

## 1d) Apply your model

### d1) Load Sequences (1.5)
Write a function `load_sequences(start, end)` that extracts the "finished paths" from the data published in https://snap.stanford.edu/data/wikispeedia.html, does preprocessing, cleans up all potential files and returns the transitions as a list of sequences. Disregard the rest of the information like ip_hash, time... The start and end variable specify a starting token and an ending token.


Do all of this task in code (i.e. download (using the requests library), unzip, read and process the file). Make sure you **read the comments** at the beginning of the `paths_finished.tsv` file. Do **not** download the wikipedia articles along. After you are done, remove all potential files that you created during the process. There should be 1043 univariate sequences for `start, end = ('Asteroid', 'Viking')`.

### d2) Eval Dataset (1)
Fit a first order and second order model to the sequences from `'Asteroid'` to `'Viking'`. According to AIC and BIC, which model fits the data better?  Make a clear statement (as text), which model is to be preferred. Store that string in the variable `better_model`, also write that string to `better_model.txt`. Make sure you mention the AIC and BIC scores you obtained in the text.

In [10]:
import requests
import tarfile
import tempfile
import pandas as pd
import os

def load_wikipaths():
    url = 'https://snap.stanford.edu/data/wikispeedia/wikispeedia_paths-and-graph.tar.gz'
    with requests.get(url) as resp:
        with tempfile.NamedTemporaryFile(delete=False) as f:
            f.write(resp.content)
            name = f.name
        with tarfile.open(name) as tar:
            tsv = tar.extractfile('wikispeedia_paths-and-graph/paths_finished.tsv')
            df = pd.read_csv(tsv, sep='\t', skiprows=16, header=None, 
                    names='hashedIpAddress timestamp durationInSec path rating'.split())
            os.unlink(name)
    return df

df = None

In [11]:
def process_back_clicks(sequence):
    stack = []
    history = []
    for c in sequence:
        if c != '<':
            stack.append(c)
            history.append(c)
        else:
            stack.pop()
            history.append(stack[-1])
    return history

def load_sequences(start, end):
    global df
    if df is None:
        df = load_wikipaths()
    
    
    return [process_back_clicks(s) for s in df.path.str.split(';').tolist() 
                            if s[0] == start and s[-1] == end]

In [12]:
sequences1 = load_sequences('Asteroid', 'Viking')
sequences2 = load_sequences('Asteroid', 'Viking')

t_m1, row_names1, col_names1 = fit_mc(sequences1, 1)
t_m2, row_names2, col_names2 = fit_mc(sequences2, 2)

aic1 = AIC(sequences1, t_m1, row_names1, col_names1)
aic2 = AIC(sequences2, t_m2, row_names2, col_names2)
bic1 = BIC(sequences1, t_m1, row_names1, col_names1)
bic2 = BIC(sequences2, t_m2, row_names2, col_names2)

better_model = f'1-order model is better: AIC1={aic1:.2f} BIC1={bic1:.2f} vs AIC2={aic2:.2f} BIC2={bic2:.2f}'

with open('better_model.txt', 'w') as f:
    f.write(better_model)

# Task 2 Graphs (7 points total)

In this task you are requested to do some basic tasks on graphs.

## 2a) Building a graph (1)


Represent the sequences for `start, end = ("Apollo", "Skiing")` from previous task d) as a directed graph in networkx. Use one of the standard networkx drawing functions. Exclude the RESET states. Show the plot here in the notebook and also save it as `apollo_skiing.png`. Include the direction and the node labels. Make sure the labels are at least somewhat readable. Argue whether this is a good visualization. Save this argument in a variable `apollo_skiing` as a string. Also write it to a file `apollo_skiing.txt`.


## 2b) Classic link prediction metrics (0.5+0.5+1)

In this task you should compute different metrics for link prediction. Do not use functions from networkx. Assume the graph is undirected.

Write the functions `Jaccard_link(A)`, `Adamic_Adar_link(A)` and `Katz_link(A, beta)`.
The parameter A is a binary adjancency matrix (assume a 2d-numpy array, it can be dense or sparse), beta is defined in the lecture. They both return a matrix with the corresponding scores for all the nodes in the network.
Help for Katz: Search for geometric series of matrices ;). `raise` a ValueError if the provided `beta` is to large.

The package networkx contains the Zachary’s Karate Club graph. It also contains functions that compute link scores. You can use it for testing.

## 2c) Evaluate link prediction with mean inverse rank (1.5)

To evaluate your link prediction we will use the mean inverse rank (mir). The basic idea is, that edges that really exist should be ranked higher by your metric in comparison with edges that do not exist.

Starting from train/test split of the edges you rank the missing edges according to the metric in descending order. Now for all the heldout edges you obtain the place at which this ground truth edge is in the list of edges. This is the rank of the edge. You then calculate the mean of the inverse rank (1/rank) of all heldout edges.

Write a function `cross_val_mir(G, func)` that computes the mean inverse rank for function `func` assume that the function takes a binary adjacency matrix and no other parameter. G is an undirected networkx graph. Use 10 fold cross validation on the edge set. Use `np.array_split` to split the edges in G into the 10 chunks. Return 1) the average mean inverse rank and 2) the standard deviation of the mir.

Apply this function to the three link prediction metrics you previously defined. Use the Zachary Karate Club graph as provided in the networkx package. Store the results as a dict 
```python
{"Jaccard" : 0.0,
 "Adamic" : 0.0,
 "Katz" : 0.0}
```
in `results_2c`. Store the beta you used for `Katz` in the variable `beta_2c`.

## 2c) Random walks on a graph (1)

Write a function `random_walks(A, start_node, length, rng)` that yields random walks of length `length` starting at node_index `start_node` on a graph given by a binary adjacency matrix `A`. Assume the graph is undirected. At each node the walker walks to another node at random. Do not use any restart conditions. At any node: Determine the k nodes to pick next, then choose the node which will be visited next with a call to `rng.randint(0,k)`.



## 2d) Node2vec (1.5)

Using your function `random_walks` along with the gensim.models.Word2Vec function to train a node2vec model.
Write a helper function `get_corpus(A, length, n_per_node, rng)` that takes the binary adjacency matrix and creates a corpus (list of sequence of integers). It thereby uses the random walks function previously defined. To obtain `n_per_node` univariate sequences for each node. Go through the nodes in ascending order. Finally shuffle the list of sequences obtained using `np.permutation`. Return that sequence.

Thereafter write a function `node2vec(G, dimensions, length, n_per_node, rng)` that trains Word2Vec embeddings of a specified number of dimensions for the networkx-graph G. It first creates a corpus for graph G, using the helper function. It then relabels the corpus such that the entries are the same as in the graph. It then uses gensim.models.word2vec to train a node embedding. Use default arguments for word2vec unless specified otherwise. (Only provide the corpus and the dimensions). Return the trained Word2Vec model.

## 2e) Word2Vec/Node2Vec vs Glove (1)

Explain the differences and similarities of the Anything2vec approach and Glove. Save your explanation as a string in the variable `glove`. Also write it to a file `glove.txt`

In [ ]:
import networkx as nx